# 4.8 Grouping Data and Aggregating Variables

## Setup

In [34]:
# Import Libraries

import pandas as pd
import numpy as np
import os

In [35]:
# Import ords_prods_merge

path = r'C:\Users\lefev\Documents\Data Analytics Course\Instacart Basket Analysis'

In [36]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_prods_merge_updated.pkl'))

## Find the average 'order_number' per deparment_id

In [38]:
# Aggregate the mean of 'order_number' (number of orders) column grouped by 'department_id'

ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


## The answers for the entire dataframe are not so different from the sample size in the exercise. Typically on +/- 1 or 2 orders

## Create a loyalty flag for existing customers

In [41]:
# Create a max order column

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\lefev\AppData\Local\Temp\ipykernel_16316\4155048890.py:3: FutureWarning: The provided callable <function max at 0x0000029E4D77A0C0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [42]:
# Create a loyalty flag

ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [43]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [44]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [45]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

## Question 5 - How does each loyalty catagory spend on products

In [47]:
# Frist group by loyalty catagory 

ords_prods_merge.groupby('loyalty_flag')['prices'].describe()

,count,mean,std,min,25%,50%,75%,max
loyalty_flag,,,,,,,,
Loyal customer,10284093.0,10.386336,328.017787,1.0,4.2,7.4,11.2,99999.0
New customer,6243990.0,13.294670,597.560299,1.0,4.2,7.4,11.3,99999.0
Regular customer,15876776.0,12.495717,539.720919,1.0,4.2,7.4,11.3,99999.0


## Loyal customers do tend to buy slightly cheaper products than new and regular customers. The newer customers spend the most on products

## Question 6 - Create a spending flag for Low Spender and High Spender 

In [64]:
# Create a new avg_price column using groupby and transform 

ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform('mean')

In [66]:
# Create spending flag

ords_prods_merge.loc[ords_prods_merge['avg_price'] > 10, 'spending_flag'] = 'High spender'

In [68]:
ords_prods_merge.loc[ords_prods_merge['avg_price'] <= 10, 'spending_flag'] = 'Low spender'


In [70]:
ords_prods_merge['spending_flag'].value_counts(dropna=False)

spending_flag
Low spender     31770995
High spender      633864
Name: count, dtype: int64

## Question 7 - Create an order frequency flag marking regularity of user ordering behavior

In [73]:
# Create a new median_days column using groupby and transform

ords_prods_merge['median_days'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform('median')

In [75]:
# Create the frequency_flag 

ords_prods_merge.loc[ords_prods_merge['median_days'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [76]:
ords_prods_merge.loc[(ords_prods_merge['median_days'] > 10) & (ords_prods_merge['median_days'] <= 20), 'frequency_flag'] = 'Regular customer'

In [77]:
ords_prods_merge.loc[ords_prods_merge['median_days'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [83]:
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: count, dtype: int64

# Export

In [88]:
# Export 

ords_prods_merge.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_prods_merge_updated_grouped.pkl'))